In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import utils
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn import metrics
import warnings
from sklearn.metrics import confusion_matrix

In [2]:
warnings.filterwarnings("ignore")
root_path = r"F:\fmri\fmri-new\machine\aal_NYU_10_24"
cc=216
weidu = int(cc*cc/2-cc/2)

data = pd.read_csv("new_data.csv")

In [3]:
warnings.filterwarnings("ignore")

cc=216
weidu = int(cc*cc/2-cc/2)

def fea(aalFeaPer,aalFeaSel,aalFeaRid,p,s,r,allnum):
    feature_per = feaSel(aalFeaPer,allnum,p)
    feature_sel = feaSel(aalFeaSel,allnum,s)
    feature_rid = feaSel(aalFeaRid,allnum,r)
    features = set(feature_per) & set(feature_sel) & set(feature_rid)
    return features,len(features)

def feaSel(aalFeaPer,bigW,pinlv):
    aalFeaPer.value_counts()
    lie = list(aalFeaPer.values)
    feature_per = []
    for i in range(1, bigW+1):
        if lie[i - 1] >= pinlv:
            feature_per.append(i)
    return feature_per

def k_fold_split(dataset,k_fold,k):
    k_sample_count = dataset.shape[0]//k_fold
    test_begin = k_sample_count*k
    test_end = k_sample_count*(k+1)
    test_data = dataset[test_begin:test_end]
    train_data = pd.concat([dataset[:test_begin],dataset[test_end:]])
    return train_data,test_data

def data_spilt(dataset):
    dataset = dataset.drop("SUB_ID",axis=1)
    ill_data = dataset[dataset["label"]==1]
    ill_data = utils.shuffle(ill_data,random_state=0)
    unill_data = dataset[dataset["label"]==2]
    unill_data = utils.shuffle(unill_data,random_state=0)
    return ill_data,unill_data

def k_data(ill_data,unill_data,k_fold,k,weidu):
    train_data1, test_data1 = k_fold_split(ill_data, k_fold, k)
    train_data2, test_data2 = k_fold_split(unill_data, k_fold, k)
    train_data = pd.concat([train_data1, train_data2])
    test_data = pd.concat([test_data1, test_data2])
    train_data = utils.shuffle(train_data, random_state=0)
    test_data = utils.shuffle(test_data, random_state=0)
    traindata_dummies = pd.get_dummies(train_data)
    # print("Features after get_dummies:\n",list(traindata_dummies.columns))
    train_features = traindata_dummies.iloc[:,:weidu]
    X_train = train_features.values
    y_train = traindata_dummies.iloc[:,-1].values
    testdata_dummies = pd.get_dummies(test_data)
    test_features = testdata_dummies.iloc[:,:weidu]
    X_test = test_features.values
    y_test = testdata_dummies.iloc[:,-1].values
    # print(np.shape(X_test))
    return X_train,X_test,y_train,y_test

def select_fun(select,X_train,X_test,y_train):
    select.fit(X_train,y_train)
    X_train1 = select.transform(X_train)
    # print(np.shape(X_train1))
    X_test1 = select.transform(X_test)
    return X_train1,X_test1

def feadata(aal_fea):
    aal_fea = list(map(str, aal_fea))
    aal_fea.insert(0, 'SUB_ID')
    aal_fea.insert(len(aal_fea),'label')
    data = pd.DataFrame(raw_data, columns=aal_fea)
    return data

def trainTest(data,weidu):
    test_score = []
    train_score = []
    precision = []
    recall = []
    f1 = []
    auc = []
    spec = []
    ill_data, unill_data = data_spilt(data)
    for k in range(k_fold):
        X_train, X_test, y_train, y_test = k_data(ill_data, unill_data, k_fold, k,weidu)
        # print("=========================================================")
        # print('逻辑回归')
        model = LogisticRegression(max_iter=1000, random_state=0, class_weight={1: 0.65, 2: 0.35})
        # print('随机森林')
        # model = RandomForestClassifier(n_estimators=35, max_depth=7, random_state=0)
        # print('svm')
        # model = SVC(gamma=0.1, C=10, class_weight={1: 0.65, 2: 0.35})
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        # print("y_predict",y_predict)
        # print("y_test",y_test)
        train_score.append(model.score(X_train, y_train))
        test_score.append(model.score(X_test, y_test))
        precision.append(precision_score(y_test, y_predict, average='binary'))
        recall.append(recall_score(y_test, y_predict, average='binary'))
        f1.append(f1_score(y_test, y_predict, average='binary'))
        tn,fp,fn,tp = confusion_matrix(y_test,y_predict).ravel()
        spec.append(tn/(tn+fp))
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict, pos_label=2)
        auc.append(metrics.auc(fpr, tpr))
    train_acc = np.mean(train_score)
    test_acc = np.mean(test_score)
    pre = np.mean(precision)
    rec = np.mean(recall)
    f1_sc = np.mean(f1)
    sp = np.mean(spec)
    auc_score = np.mean(auc)
    return train_acc, test_acc, pre, rec, f1_sc, sp,auc_score

def resHanshu(s,r,weidu):
    # print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    aal_fea, weiduF = fea(aalFeaPer, aalFeaSel, aalFeaRid, 10, s, r,weidu)
    print("s:", s, "r:", r, weiduF)
    data = feadata(aal_fea)
    train_acc, test_acc, pre, rec, f1_sc, sp,auc_score = trainTest(data,weiduF)
    return train_acc, test_acc, pre, rec, f1_sc, sp,auc_score,weiduF

In [4]:
aalFeaPer = pd.read_csv("feature/PerPinlv.csv", header=None)
aalFeaSel = pd.read_csv("feature/SelPinlv.csv", header=None)
aalFeaRid = pd.read_csv("feature/ridPinlv.csv", header=None)

raw_data = pd.read_csv("new_data.csv")

k_fold = 10

In [5]:
Train_ACC, Test_ACC, Precision, Recall,F1,SP,AUC,weiduMatrix=resHanshu(1,8,weidu)
print(Test_ACC)

s: 1 r: 8 46
0.93125
